# 堆排序 

## topk问题 

1. 现在有n个数，设计算法得到的前k大的数。（k<n）
2. 解决思路：
    - 排序后切片：$O(nlogn)+k-->O(nlogn)$ <br>上亿数据量很慢
    - 排序LowB三人组：$O(kn)$ <br> 如冒泡排序，只需要排序前k趟；插入排序：维护前100个数，来一个数判断是否插入；选择排序：选择100次，每次选最大的数跟第一个交换
    - 堆排序思路： $O(nlogk)$
        - 1. 取列表前k个元素建立一个小根堆。堆顶就是目前第k大的数。
        - 2. 依次向后遍历原列表，对于列表中的元素，如果小于堆顶，则忽略该元素；如果大于堆顶，则将堆顶更换为该元素，并且对堆进行一次调整。
        - 3. 遍历列表所有元素后，倒序弹出堆顶。

ps: 目前讲的LowB和NB三人组实际上都是比较排序，通过比较数字大小判断是否需要交换

In [43]:
# 小根堆
def sift(li, low, high):
    i = low
    j = 2 * i + 1
    tmp = li[low]
    while j <= high:
        if j + 1 <= high and li[j+1] < li[j]:
            j = j + 1
        if li[j] < tmp:
            li[i] = li[j]
            i = j
            j = 2 * i + 1
        else:
            break
        li[i] = tmp

In [32]:
def heap_sort(li):
    n = len(li)
    # 堆的向下调整，建堆
    for i in range((n-2)//2, -1, -1):
        # i表示建堆的时候调整的部分的根的下标
        sift(li, i, n-1)
    # 挨个出数，按顺序
    for i in range(n-1, -1, -1):
        # i 指向当前堆的最后一个元素
        li[0], li[i] = li[i], li[0]
        sift(li, 0, i-1) #i-1是新的high

In [44]:
def topk(li, k):
    heap = li[0:k]
    #建堆
    for i in range((k-2)//2, -1, -1):
        sift(heap, i, k-1)
    #遍历
    for i in range(k, len(li)-1):
        if li[i] > heap[0]:
            heap[0] = li[i]
            sift(heap, 0 ,k-1)
    #出数
    for i in range(k-1, -1, -1):
        heap[0], heap[i] = heap[i], heap[0]
        sift(heap, 0, i-1)
    return heap

In [47]:
import random
li = list(range(1000))
random.shuffle(li)

In [48]:
topk(li, 10)

[999, 998, 997, 996, 995, 994, 993, 992, 991, 990]

# 归并排序<br>(非原地排序，需要额外空间)

## 归并 

1. 假设现在的列表分两段有序，如何将其合成为一个有序列表

In [66]:
def merge(li, low, mid, high):
    i = low
    j = mid + 1
    ltmp = []
    while i <= mid and j <= high: #只要左右两边都有数
        if li[i] < li[j]:
            ltmp.append(li[i])
            i += 1
        else:
            ltmp.append(li[j])
            j += 1
        # while执行完，肯定有一部分没数了
        while i <= mid:
            ltmp.append(li[i])
            i += 1
        while j <= high:
            ltmp.append(li[j])
            j += 1
        li[low:high+1] = ltmp

In [52]:
li = [2,4,5,7,1,3,6,8]
merge(li,0,3,7)

In [53]:
li

[1, 2, 4, 5, 7, 3, 6, 8]

## 使用归并

1. 分解：将列表越分越小，直至分成一个元素。
2. 终止条件：一个元素是有序的。
3. 合并：将两个有序列表归并，列表越来越大。

In [67]:
# 先分解后归并的递归，类似汉诺塔
def merge_sort(li, low, high):  
    if low < high: # 至少有两个元素，递归
        mid = (low + high)//2
        merge_sort(li, low, mid)
        merge_sort(li, mid + 1, high)
        merge(li, low, mid, high)
        #print(li[low:high+1])

In [72]:
li = list(range(10))
import random
random.shuffle(li)
print(li)

[5, 1, 7, 3, 4, 9, 6, 0, 8, 2]


In [73]:
merge_sort(li, 0, len(li)-1)
np.transpose(li)

[5, 1]
[5, 1, 7]
[3, 4]
[5, 1, 7, 3, 4]
[9, 6]
[9, 6, 0]
[8, 2]
[9, 6, 0, 8, 2]
[5, 1, 7, 3, 4, 9, 6, 0, 8, 2]


array([5, 1, 7, 3, 4, 9, 6, 0, 8, 2])

## 时间复杂度 

$O(nlogn)$:<br>
每层运行n次，一共log(n)层，折半

## 空间复杂度

$O(n)$:<br>
存在临时空间ltmp，使用空间O(n)

python的sort方法，结合归并排序和插入排序，做了优化

# NB三人组小结 

1. 三种排序算法的时间复杂度都是$O(nlogn)$
2. 一般情况下，就运行时间而言：
    - 快速排序 < 归并排序 < 堆排序
3. 三种排序算法的缺点：‘
    - 快速排序：极端情况下排序效率低
    - 归并排序：需要额外的内存开销
    - 堆排序：在快的排序算法中相对较慢

排序算法总结：https://blog.csdn.net/zxzxzx0119/article/details/79826380 <br>
快排的空间复杂度不是$O(1)$，递归需要用系统空间：函数每走一层，需要存将函数存一个位置。<br>
稳定性：当两个元素值一样时，保证它们的相对位置不变。<br>

In [74]:
# 稳定性即保证下列两个name相同的字典位置不变。
{'name':'a','age':18}
{'name':'b','age':20}
{'name':'a','age':25}

{'name': 'a', 'age': 25}

有顺序挨个移动的稳定，跳着换的不稳定

# 希尔排序 

1. 希尔排序（Shell Sort）是一种<b>分组插入排序</b>算法。
2. 首先取一个整数 $d_1 = n/2$，将元素分为$d_1$个组，每组相邻两元素之间距离为$d_1$，在各组内进行直接插入排序；
3. 取第二个整数$d_2 = d_1/2$，重复上述分组排序过程，直到$d_1 = 1$，即所有元素在同一组内进行直接插入排序。
4. 希尔排序每趟并不使某些元素有序，而是使整体数据越来越接近有序；最后一趟排序使得所有数据有序。

In [75]:
def insert_sort_gap(li, gap): # 分组距离
    for i in range(gap, len(li)): # i表示摸到的牌的下标
        tmp = li[i]
        j = i - gap # j指的是手里的牌的下标
        while j >= 0 and li[j] > tmp:
            li[j+gap] = li[j]
            j -= gap
        li[j+gap] = tmp

In [76]:
def shell_sort(li):
    d = len(li) // 2
    while d >= 1:
        insert_sort_gap(li, d)
        d //= 2

In [81]:
li = list(range(100))
random.shuffle(li)

In [82]:
shell_sort(li)
np.transpose(li)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

## 时间复杂度 

希尔排序的时间复杂度讨论比较复杂，并且和选取的gap序列有关。
https://en.wikipedia.org/wiki/Shellsort#Gap_sequences

# 计数排序 

对列表进行排序，已知列表中的数范围都在0到100之间。设计时间复杂度为O(n)的算法。

In [101]:
def count_sort(li, max_count=100):
    count = [0 for _ in range(max_count+1)]
    for val in li:
        count[val] += 1 # n次
    li.clear()
    for ind, val in enumerate(count):
        for i in range(val):
            li.append(ind) #共append n次，整个循环加起来为O(n)
            # print(ind,val)

In [102]:
li = [random.randint(0,100) for _ in range(100)]
np.transpose(li)

array([ 60,  69,  76,  47,  48,  74,  76,  70,  23,  82,   2,  47,  73,
        69,  54,  97,   4,  34,  26,  96,  76,  94,  80,  70,   7,  76,
        36,  11,  82,  92,  54,  35,  49, 100,  69,  10,  28,  60,  81,
        58,  21,  57,  70,  53,  71,  95,  68,  35,  49,  12,  56,   2,
        10,  39,  12,   8,  44,  14,  56,  89,  68,  85,  33,  99,  51,
        15,  97,  39,  17,   0,  28, 100,  27,  29,  15,  70,  60,  63,
        24,  99,  12,  20,  14,   4,  55,  95,  18,  10,  85,  52,  44,
        28,  28, 100,  17,  20,  24,  85,  68,  28])

In [103]:
count_sort(li)
np.transpose(li)

0 1
2 2
2 2
4 2
4 2
7 1
8 1
10 3
10 3
10 3
11 1
12 3
12 3
12 3
14 2
14 2
15 2
15 2
17 2
17 2
18 1
20 2
20 2
21 1
23 1
24 2
24 2
26 1
27 1
28 5
28 5
28 5
28 5
28 5
29 1
33 1
34 1
35 2
35 2
36 1
39 2
39 2
44 2
44 2
47 2
47 2
48 1
49 2
49 2
51 1
52 1
53 1
54 2
54 2
55 1
56 2
56 2
57 1
58 1
60 3
60 3
60 3
63 1
68 3
68 3
68 3
69 3
69 3
69 3
70 4
70 4
70 4
70 4
71 1
73 1
74 1
76 4
76 4
76 4
76 4
80 1
81 1
82 2
82 2
85 3
85 3
85 3
89 1
92 1
94 1
95 2
95 2
96 1
97 2
97 2
99 2
99 2
100 3
100 3
100 3


array([  0,   2,   2,   4,   4,   7,   8,  10,  10,  10,  11,  12,  12,
        12,  14,  14,  15,  15,  17,  17,  18,  20,  20,  21,  23,  24,
        24,  26,  27,  28,  28,  28,  28,  28,  29,  33,  34,  35,  35,
        36,  39,  39,  44,  44,  47,  47,  48,  49,  49,  51,  52,  53,
        54,  54,  55,  56,  56,  57,  58,  60,  60,  60,  63,  68,  68,
        68,  69,  69,  69,  70,  70,  70,  70,  71,  73,  74,  76,  76,
        76,  76,  80,  81,  82,  82,  85,  85,  85,  89,  92,  94,  95,
        95,  96,  97,  97,  99,  99, 100, 100, 100])

## 时间复杂度 

$O(n)$

In [105]:
from cal_time import *

ModuleNotFoundError: No module named 'cal_time'

In [104]:
@cal_time
def sys_sort(li):
    li.sort()

NameError: name 'cal_time' is not defined

## 计数排序有限制 

1. 需要在0-100之间
2. 需要消耗大量内存（0-100则100）
3. 对小数等其他形式元素计数复杂

# 桶排序 

1. 在计数排序中，如果元素的范围比较大（比如在1到1亿之间），如何改造算法？
2. 桶排序（Bucket Sort）：首先将元素分在不同的桶中，在对每个桶中的元素排序。

In [14]:
def bucket_sort(li, n=100, max_num=10000):
    # n是分多少个桶，max_num是数值上仙
    buckets = [[] for _ in range(n)] #创建桶列表
    for var in li:
        i = min(var // (max_num // n), n-1) # i 表示var放到第几号桶里
        #0->0, 186->1
        buckets[i].append(var)
    # 对每个桶排序，保持桶内顺序
    for j in range(len(buckets[i])-1, 0, -1):
        #[0,2,4,3]冒泡-->[0,2,3,4]
        if buckets[i][j] < buckets[i][j-1]:
            buckets[i][j], buckets[i][j-1] = buckets[i][j-1], buckets[i][j]
        else:
            break
    sorted_li = []
    for buc in buckets:
        sorted_li.extend(buc)
    return sorted_li

In [15]:
import random
import numpy as np

In [18]:
li = [random.randint(0,10000) for i in range(100000)]
np.transpose(li)

array([7513, 1361, 9034, ..., 6602, 2356, 1633])

In [20]:
li = bucket_sort(li)
np.transpose(li)

array([  59,   88,   69, ..., 9924, 9981, 9989])

## 讨论 

1. 桶排序的表现取决于数据的分布。也就是需要对不同数据排序时采取不同的分桶策略。
2. 平均情况时间复杂度：O(n+k)  (n是一共多少个数，k是桶的个数)
3. 最坏情况时间复杂度：$O(n^2k)$
4. 空间复杂度：O(nk)

# 基数排序 

1. 多关键字排序：假如现在有一个员工表，要求按照薪资排序，年龄相同的员工按照年龄排序。
    - 先按照年龄进行排序，再按照薪资进行稳定的排序。

- 对32，13，94，52，17，54，93排序，是否可看作多关键字排序？
    - 先按照各位进行排序，再按照十位进行排序。

In [1]:
def radix_sort(li):
    max_num = max(li) # 最大值 99->2, 888->3, 10000->5
    it = 0
    while 10 ** it <= max_num: #判断有多少位数，执行多少次循环
        buckets = [[] for _ in range(10)]
        for var in li:
            # 987 it=1 987//10->98 98%10->8; it=2 987//100->9 9%10->9
            digit = (var // 10 ** it) % 10
            buckets[digit].append(var)
        # 分桶完成
        li.clear()
        # 把数重新写回li
        for buc in buckets:
            li.extend(buc)
        it += 1

In [2]:
import random

In [3]:
li = list(range(1000))
random.shuffle(li)
radix_sort(li)
print(li)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

代码2

In [4]:
def list_to_buckets(li, base, iteration):
    buckets = [[] for _ in range(base)]
    for number in li:
        digit = (number // (base ** iteration)) % base
        buckets[digit].append(number)
    return buckets

In [5]:
def buckets_to_list(buckets):
    return [x for bucket in buckets for x in bucket]

In [6]:
def radix_sort(li, base=10):
    maxval = max(li)
    it = 0
    while base ** it <= maxval:
        li = buckets_to_list(list_to_buckets(li, base, it))
    return li

## 复杂度

1. 时间复杂度：$O(kn)$
2. 空间复杂度：$O(k+n)$
3. k表示数字位数

快排和基数排序比较：
- 快排受列表中元素个数影响，而基数排序受max_num的位数影响（数字的范围）

## 字符串排序 

abcd和ab
- abcd  ab00，在不够位数的字符后面统一加比所有字符都小的数